# Load Dependencies

In [10]:
from sqlalchemy import create_engine
import pandas as pd
import psycopg2
from sqlalchemy_utils import create_database, database_exists, drop_database #pip install sqlalchemy-utils

ModuleNotFoundError: No module named 'config'

# Create Database

In [2]:
connection_str ="postgres:PW@localhost:5432/suicide_db"
engine = create_engine(f'postgresql://{connection_str}')

# If a PostgreSQL database with this name exists
if database_exists(engine.url):
    # Delete PostgreSQL database 
    drop_database(engine.url)
    # Create empty PostgreSQL database
    create_database(engine.url)
# Otherwise
else:
    # Create empty PostgreSQL database
    create_database(engine.url)
    

# Create Tables

In [3]:
def create_tables():
    """ create tables in the PostgreSQL database"""
    commands = (
                """        
                CREATE TABLE "gdp"(
                "country" TEXT,
                "year" INT,
                "gdp_per_capita" FLOAT,
                PRIMARY KEY ("country","year")
                );
                """,
                """
                CREATE TABLE "hdi"(
                "hdi_rank" INT,
                "country" TEXT,
                "year" INT,
                "value" FLOAT,
                PRIMARY KEY ("country","year")
                );
                """,
                """
                CREATE TABLE "suicide"(
                "country" TEXT,
                "country_code" TEXT,
                "year" INT,
                "suicide_rate" FLOAT,
                PRIMARY KEY ("country","year")
                );
                """
    )
    conn= None

    try:
        # connect to the PostgreSQL server
        engine = create_engine(f'postgresql+psycopg2://{connection_str}')
        conn = engine.connect()
        # create table one by one
        for command in commands:
            trans = conn.begin()
            conn.execute(command)
            trans.commit()
        # Close connection
        tables = engine.table_names()
        conn.close()
        print (f'The tables: {tables} were created')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
create_tables()        

The tables: ['gdp', 'hdi', 'suicide'] were created


<ipython-input-3-e9cf2ce35dba>:43: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  tables = engine.table_names()


# Load Clean Data

In [4]:
gdp_file = '../Resources/gdp_clean_df.csv'
gdp_df = pd.read_csv(gdp_file)

hdi_file = '../Resources/HDI_clean_df.csv'
hdi_df = pd.read_csv(hdi_file)

suicide_file = '../Resources/suicide_clean_df.csv'
suicide_df = pd.read_csv(suicide_file)

# Save Data into Tables

In [5]:
connection_str ="postgres:PW@localhost:5432/suicide_db"
engine = create_engine(f'postgresql://{connection_str}')

In [6]:
engine.table_names()

<ipython-input-6-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['gdp', 'hdi', 'suicide']

In [7]:
suicide_df.to_sql(name='suicide', con=engine, if_exists='append', index=False)


In [8]:
gdp_df.to_sql(name='gdp', con=engine, if_exists='append', index=False)

In [9]:
hdi_df.to_sql(name='hdi', con=engine, if_exists='append', index=False)